# Exploration of FRP parameter space using BO and KMC

In [1]:
import torch
from KMC import KMC
import numpy as np
import pandas as pd
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.acquisition.objective import ConstrainedMCObjective
from botorch.optim import optimize_acqf
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.utils.sampling import draw_sobol_samples
from botorch.models.transforms.outcome import Standardize
from botorch.utils.transforms import normalize, unnormalize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.double

## functions used to call KMC simulation

In [2]:
def evaluate(x):
    ki = 10 ** float(x[0])
    kia = 5.0e3
    kib = 5.0e3
    kpaa = 10 ** float(x[1])
    kpbb = 10 ** float(x[2])
    r1 = 10 ** float(x[3])
    r2 = 10 ** float(x[4])
    kpab = kpaa/r1
    kpba = kpbb/r2
    ktaa = 10 ** float(x[1] + x[5])
    ktbb = 10 ** float(x[2] + x[6])
    ktab = (ktaa * ktbb) ** 0.5
    ktraa = 2 * kpaa * (10 ** float(x[7]))
    ktrab = 2 * kpab * (10 ** float(x[8]))
    ktrba = 2 * kpba * (10 ** float(x[9]))
    ktrbb = 2 * kpbb * (10 ** float(x[10]))
    ratio = 2 * (10 ** float(x[11]))
    conc = 10 ** float(x[12])

    params = [ki,kia,kib,kpaa,kpab,kpba,kpbb,ktab,ktaa,ktbb,ktraa,ktrab,ktrba,ktrbb,ratio,conc]
    Mn, MWD, conv = KMC(params)
    return Mn, MWD, conv

def batch_evaluate(x_list):
    results = np.array([evaluate(x) for x in x_list])
    Mn_batch, MWD_batch, conv_batch = torch.tensor(results.T).to(device, dtype)
    obj = (1 / MWD_batch).unsqueeze(-1)  # objective: reciprocal of molecular weight distribution (minimization)
    con1 = (1000 - Mn_batch).unsqueeze(-1)  # constraint1: molecular weight > 1000
    con2 = (0.6 - conv_batch).unsqueeze(-1)  # constraint2: conversion > 0.6
    return obj, con1, con2

## parameters, hyperparameters and functions for BO

In [3]:
# set the value ranges of the parameters
bounds = torch.tensor([[-6,-2,-2,-2,-2,1,1,-5,-5,-5,-5,-3,-1], [-4,5,5,2,2,7,7,-2,-1,-1,-2,-1,1]], device=device, dtype=dtype)
standard_bounds = torch.zeros(2, 13, device=device, dtype=dtype)
standard_bounds[1] = 1

In [4]:
# define the batch_size and the hyperparameters
DEMO_TEST = True
BATCH_SIZE = 5
NUM_RESTARTS = 10 if not DEMO_TEST else 2
RAW_SAMPLES = 512 if not DEMO_TEST else 32
N_BATCH = 200 if not DEMO_TEST else 5
MC_SAMPLES = 128 if not DEMO_TEST else 16

In [5]:
# function for establishing/updating GP models
def establish_model(train_x, obj, con1, con2):
    train_x = normalize(train_x, bounds)
    # three GP models for MWD, Mn and conversion
    model_obj = SingleTaskGP(train_x, obj, outcome_transform=Standardize(m=1))
    model_con1 = SingleTaskGP(train_x, con1, outcome_transform=Standardize(m=1))
    model_con2 = SingleTaskGP(train_x, con2, outcome_transform=Standardize(m=1))

    model = ModelListGP(model_obj, model_con1, model_con2)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [6]:
# acquisition function for proposing new candidates
def optimize_acqf_and_get_observation(acq_func):
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
    )
    # observe new values 
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj, new_con1, new_con2 = batch_evaluate(new_x)
    return new_x, new_obj, new_con1, new_con2

In [7]:
# define a feasibility-weighted objective for optimization
def obj_callable(Z):
    return Z[..., 0]

def constraint1_callable(Z):
    return Z[..., 1]

def constraint2_callable(Z):
    return Z[..., 2]

constrained_obj = ConstrainedMCObjective(
    objective=obj_callable,
    constraints=[constraint1_callable, constraint2_callable],
)

## main program

In [8]:
train_x = draw_sobol_samples(bounds=bounds, n=1, q=28, seed=2023).squeeze(0)
train_obj, train_con1, train_con2 = batch_evaluate(train_x)
mll, model = establish_model(train_x, train_obj, train_con1, train_con2)

In [9]:
# run N_BATCH rounds of BayesOpt after the initial random batch
for iteration in range(N_BATCH):

    # fit the models
    fit_gpytorch_model(mll)

    # define the qNEI and qNEI acquisition modules using a QMC sampler
    qmc_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)

    # for best_f, we use the best observed noisy values as an approximation
    qNEI = qNoisyExpectedImprovement(
        model=model,
        X_baseline=normalize(train_x, bounds),
        sampler=qmc_sampler,
        objective=constrained_obj,
    )

    # optimize and get new observation
    new_x, new_obj, new_con1, new_con2 = optimize_acqf_and_get_observation(qNEI)

    # update training points
    train_x = torch.cat([train_x, new_x])
    train_obj = torch.cat([train_obj, new_obj])
    train_con1 = torch.cat([train_con1, new_con1])
    train_con2 = torch.cat([train_con2, new_con2])

    # reinitialize the models so they are ready for fitting on next iteration
    # use the current state dict to speed up fitting
    mll, model = establish_model(train_x, train_obj, train_con1, train_con2)

    print('iteration %d' % iteration)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4


## save data and model

In [10]:
df = pd.DataFrame(train_x.cpu().detach().numpy())
df['obj'] = train_obj.cpu().detach().numpy()
df['con1'] = train_con1.cpu().detach().numpy()
df['con2'] = train_con2.cpu().detach().numpy()
df.to_csv('BO_demo_traindata.csv', index=False)

torch.save(model.state_dict(), 'BO_demo_model_params.pth')